## Hopf Invariants for finitely presented groups

Enter number of generators, and relations

This implementation uses free lie algebras and the BCH product. 

In [1]:
#### ENTER DATA HERE #####

#number of generators (X_1 , X_2 , .... ,X_{numGen})
numGen = 4
    
#Relations (note indicies start at 1)
rels = []
rels.append( [ [1,1] , [2,1] , [1,-1] , [3,1] ] )
rels.append( [ [1,1]  ])



#depth of Hopf invariants
depth = 3

In [2]:
from sage.algebras.lie_algebras.bch import bch_iterator
Q = VectorSpace(QQ, 1)

In [3]:
#### Initialize Dictionaries #####

L = {}
dimL = {}
V = {}
Vbases = {}
Hopf = {}
HopfStrict = {} #These are the hopf invrts that are not in prior weights
BCHrel= {}
evalBCH = {}
VxV={} #VxV[i] is the vector space for the tensor product of V[i] tensor V[i]
VxVbases= {}
dimHopf ={}
dimHopfStrict={}
HopfxHopf={}
preHopfxHopf ={}
incl ={}
cobrack = {}



In [4]:
#### Find Hopf[1] #####
print('~~~~~~~~~~~~~~ Weight', 0 , '~~~~~~~~~~~~~~~~~~')
#Initialize L[1] and variables
L[1] = LieAlgebra(QQ , numGen , step = 1)
L[1].inject_variables(verbose=False)
dimL[1] = L[1].dimension()



#Initialize V[1] and Vbases[1]
V[1] = VectorSpace(QQ , dimL[1])

Vbases[1] ={}
k=0
for i in L[1].basis().list():
    Vbases[1][i] = V[1].basis()[k]
    k = k +1 
    

    
##Find BCH relations and kernel of their evaluations
        # Find V[d] > ker ev_1 > ker ev_1 \cap ker ev_2 >......
BCHrel[1] = {}
Hopf[1] = V[1]


for s in range(len(rels)):
    #find BCH of the s^th relation
    w = rels[s][0][1]*L[1].basis().list()[rels[s][0][0]-1]
    for i in range(1,len(rels[s])):
        b = bch_iterator(w,rels[s][i][1]*L[1].basis().list()[rels[s][i][0]-1])
        w= L[1].zero()
        w= next(b)

    BCHrel[1][s]=w

                    ####print("BCH relation is :", BCHrel[1][s])    

    #Define linear functional (eval BCH(s^th)) :V[d]-->Q 
    images = []
    for i in L[1].basis().keys():
        images.append(BCHrel[1][s].coefficient(i)*Q.0 )

    evalBCH[1] = linear_transformation(V[1], Q , images)
    
    #restrict domain to previous filtration 
    evalBCH[1] = evalBCH[1].restrict_domain(Hopf[1])
    
    #find kernel
    Hopf[1] = evalBCH[1].kernel()
    
    
#print out results   
dimHopf[1]= Hopf[1].dimension()
print( '\ndimension =', dimHopf[1] , '\n' )

for i in range(Hopf[1].dimension()):
    print(' Invariant', i)
    v = Hopf[1].basis()[i]
    p = L[1].zero()
    for j in range(dimL[1]):
        c = V[1].coordinate_vector(v)[j]
        if c == 0:
            continue
        p = p + c*L[1].basis().list()[j]
    print("    ",p)


##################################
#################################





#####indexing ???????????
for d in range(2,depth+1):
    #### Find Hopf[d] #####
    print('\n~~~~~~~~~~~~~~ Weight', d -1 , '~~~~~~~~~~~~~~~~~~')
    #Initialize L[d] and variables
    L[d] = LieAlgebra(QQ , numGen , step = d)
    L[d].inject_variables(verbose=False )
    dimL[d] = L[d].dimension()
  

    #Initialize V[d] and Vbases[d]
    V[d] = VectorSpace(QQ , dimL[d])

    Vbases[d] ={}
    k=0
    for i in L[d].basis().list():
        Vbases[d][i] = V[d].basis()[k]
        k = k +1 
        
    #Construct inclusion from V[d-1] --> V[d]
    images = [V[d].basis()[k] for k in range(dimL[d-1])]
    incl[d-1] = linear_transformation(V[d-1], V[d], images)
    
    ## Construct Cobracket V[d]--> V[d-1] (X) V[d-1] ##

    #Construct tensor product
    VxV[d-1] = VectorSpace(QQ, dimL[d-1]^2)
    VxVbases[d-1] ={}

    trLb = []
    for i in range(dimL[d-1]):
        trLb.append(L[d].basis().list()[i])

    k = 0
    for j in cartesian_product([trLb, trLb]):
        VxVbases[d-1][j] = VxV[d-1].basis()[k]
        k = k+1

    #define morphism
    tempDic={k : VxV[d-1].zero() for k in L[d].basis().keys()} #these should be (dimL[d])-many elements of VxV[d-1] 

    for j in VxVbases[d-1].keys():
        p = L[d][j[0],j[1]]
        #if bracket is zero then skip to save time
        if p == 0:
            continue
        #if bracket is not zero 
        for k in L[d].basis().keys():
            tempDic[k] = tempDic[k] + p.coefficient(k)*VxVbases[d-1][j]    

    images  = list(tempDic.values())

    cobrack[d] = linear_transformation(V[d], VxV[d-1], images)
    ## Find subspace of VxV[d-1] given by Hopf[d-1] (x) Hopf[d-1]

    HopfxHopf[d-1] = []

    for k in cartesian_product([range(dimHopf[d-1]),range(dimHopf[d-1])]):
        v = Hopf[d-1].basis()[k[0]]
        w = Hopf[d-1].basis()[k[1]]

        t=0
        tens = VxV[d-1].zero()
        for i in cartesian_product([range(dimL[d-1]),range(dimL[d-1])]):
            p=V[d-1].coordinate_vector(v)[i[0]]* V[d-1].coordinate_vector(w)[i[1]]
            tens = tens + p*VxV[d-1].basis()[t]
            t=t+1

        HopfxHopf[d-1].append(tens)

    HopfxHopf[d-1] = VxV[d-1].subspace(HopfxHopf[d-1])
    
    ## Find the preimage of HopfxHopf[d-1] under cobracket
    preHopfxHopf[d-1] = cobrack[d].inverse_image(HopfxHopf[d-1])
    
    
#################
   #verification that Hopf x Hopf and preimage are working
#################    
    
#########
    ##Find BCH relations and kernel of their evaluations
            # Find V[d] > (preimage Hopf x Hopf[d-1]) > ker ev_1 > ker ev_1 \cap ker ev_2 >......
    BCHrel[d] = {}
    Hopf[d] = preHopfxHopf[d-1]

    for s in range(len(rels)):
        #find BCH of the s^th relation
        w = rels[s][0][1]*L[d].basis().list()[rels[s][0][0]-1]
        for i in range(1,len(rels[s])):
            b = bch_iterator(w,rels[s][i][1]*L[d].basis().list()[rels[s][i][0]-1])
            w= L[d].zero()
            for k in range(d):
                w = w + next(b)

        BCHrel[d][s]=w

             ####print("BCH relation is :", BCHrel[d][s])    

        #Define linear functional (eval BCH(s^th)) :V[d]-->Q 
        images = []
        for i in L[d].basis().keys():
            images.append(BCHrel[d][s].coefficient(i)*Q.0 )

        evalBCH[d] = linear_transformation(V[d], Q , images)

        #restrict domain to previous filtration 
        evalBCH[d] = evalBCH[d].restrict_domain(Hopf[d])
        #find kernel
        Hopf[d] = evalBCH[d].kernel()
        
           #####print(Hopf[d])


###########



 
    ###Find Hopf invariants in [d] but not [d-1]
    incl[d-1] = incl[d-1].restrict_domain(Hopf[d-1])
    incl[d-1] = incl[d-1].restrict_codomain(Hopf[d])
    
    priorHopf =incl[d-1].image()
    
    Q = Hopf[d]/priorHopf
    
   
    tempList = [Q.lift(v) for v in Q.basis()]
    
    HopfStrict[d] = Hopf[d].subspace(tempList)
    
    dimHopf[d]= Hopf[d].dimension()
    dimHopfStrict[d] = len(tempList)
    #display results
    
    print('\ndimension =', dimHopfStrict[d], '\n')

    for i in range(dimHopfStrict[d]):
        print(' Invariant', i)
        v = HopfStrict[d].basis()[i]
        p = L[d].zero()
        for j in range(dimL[d]):
            c = V[d].coordinate_vector(v)[j]
            if c == 0:
                continue
            p = p + c*L[d].basis().list()[j]
        print("    ",p)

~~~~~~~~~~~~~~ Weight 0 ~~~~~~~~~~~~~~~~~~

dimension = 2 

 Invariant 0
     X_2 - X_3
 Invariant 1
     X_4

~~~~~~~~~~~~~~ Weight 1 ~~~~~~~~~~~~~~~~~~

dimension = 1 

 Invariant 0
     X_24 - X_34

~~~~~~~~~~~~~~ Weight 2 ~~~~~~~~~~~~~~~~~~

dimension = 2 

 Invariant 0
     X_224 - X_234 + X_243 + X_334
 Invariant 1
     X_244 - X_344


## Evaluation of Invariants on words

The following cells will establish the Hopf pairing. It is a function that takes as input a representative of an element in $ G = F/R$ along with a Hopf invariant from above and produces a number. It does this by first finding the BCH product of the word, then multiplying the coefficents with the coefficients of the Hopf invariant (recall that the Hopf invariants are linear combinations of indicator functionals).

In [5]:
def evalHopf( weight , invariant ,word ):
    #find BCH product of the word
    d = weight
    
    l=word[0][1]*L[d].basis().list()[word[0][0]-1]
    for k in range(1, len(word)):
        b = bch_iterator(l,word[k][1]*L[d].basis().list()[word[k][0]-1])
        l = L[d].zero()
        for i in range(d):
            l = l + next(b)
        
    #import word to V[d]
    i = 0
    v = V[d].zero()
    for k in L[d].basis().keys():
        v = v + l.coefficient(k)*V[d].basis()[i]
        i = i +1
    
    #inner product of v with invariant
    out = 0
    for i in range(dimL[d]):
        out = out + invariant[i] * v[i]
    return out
    

### Example



In [6]:
## ENTER DESIRED WEIGHT
weight = 2

## SELECT THE HOPF INVARIANT FROM WEIGHT
invNum = 1

## ENTER YOUR WORD. FORMATTING FOLLOWS SAME AS RELATION
word =[[3,1] , [4,1] , [3,-1] , [4,-1] ]




#############
h = HopfStrict[weight+1].basis()[invNum]
evalHopf( weight +1 , h , word)

1/2

In [ ]:
####This is a chunk of code to verify that things are working properly 

#Verify that Hopf x Hopf [d] is correct and it's preimage
    print("\n \n \n")
    print("verification Hopf x Hopf and its preimage")
    print(f'\nThe basis for Hopf[{d-1}] is')
    
    
    for i in range(Hopf[1].dimension()):
        print(f' Vector {i}:')
        v = Hopf[1].basis()[i]
        p = L[1].zero()
        for j in range(dimL[1]):
            c = V[1].coordinate_vector(v)[j]
            if c == 0:
                continue
            p = p + c*L[1].basis().list()[j]
        print("    ",p)
    
    print(f'\nbasis for Hopf x Hopf[{d-1}] is:')
    basis = HopfxHopf[d-1].basis()
    for l in range(len(basis)): 
        print("  Vector", l , ':')
        print(basis[l])
        for k in range(dimL[d-1]^2):
            c = VxV[d-1].coordinate_vector(basis[l])[k]
            if c == 0:
                continue
            print("the coefficient of " , cartesian_product([trLb, trLb])[k] , 'is', c)
    
    print(f"\nThe basis of pre Hopf x Hpof [{d-1}] is:")
    print(preHopfxHopf[d-1])
    for i in range(preHopfxHopf[d-1].dimension()):
        print(' Vector', i)
        v = preHopfxHopf[d-1].basis()[i]
        p = L[d].zero()
        for j in range(dimL[d]):
            c = V[d].coordinate_vector(v)[j]
            if c == 0:
                continue
            p = p + c*L[d].basis().list()[j]
        print("    ",p)
    
    print("\n \n \n")